In [94]:
import plotly.offline as py
import plotly.figure_factory as ff
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
import plotly.graph_objs as go
from datetime import datetime

import numpy as np
from datetime import timedelta
import sqlite3


In [95]:
# Create your connection.
cnx = sqlite3.connect("./report.db")

trades = pd.read_sql_query(sql="SELECT * FROM trades",
                           con=cnx ,
                           parse_dates=['EntryDate', 'ExitDate']).sort_values(by=['EntryDate']).dropna()
dfs={}

def getDf(ticker: str ):
    if not ticker in dfs:
        dfs[ticker] = pd.read_sql_query('select * from ohlc_' + ticker,
                          con=cnx, parse_dates=['dt'])
    return dfs[ticker]
    
orders = pd.read_sql_query(sql="SELECT * FROM main.orders",
                           con=cnx).dropna()


In [96]:
def ann(r : pd.Series):
        return [{
            'x': r['EntryDate'],  
#             'y' : r['EntryPrice'],            
            'xref': 'x', 
            'yref': 'paper',
            'showarrow': True, 
            'xanchor': 'left',
            'text': 'buy' if r.BuySell == 1 else 'sell'
            
        },
        {
            'x': r['ExitDate'],
#             'y' : r['ExitPrice'],            
            'xref': 'x', 
            'yref': 'paper',
            'showarrow': True, 
            'xanchor': 'left',
            'text': 'sell' if r.BuySell == 1 else 'buy'
            
        }]

In [99]:

def plotOrders(startDt,endDt):
    ords=orders[startDt:endDt]
    ords[(ords.BuySell==-1) & (ords.OrderType=='Limit')].ix[:,'Price'].plot(color='red', linestyle='_', marker=">")
    ords[(ords.BuySell==1) & (ords.OrderType=='Limit')].ix[:,'Price'].plot(color='blue', linestyle='_', marker=">")


def showTrade(trdIdx):
    rec=trades.iloc[trdIdx, :]

    startDt = (rec['EntryDate'] - timedelta(minutes=320)).replace(second=0)
    endDt = (rec['ExitDate'] + timedelta(minutes=320)).replace(second=0)

    trds=trades[(trades.EntryDate > startDt) & (trades.EntryDate < endDt)]

#     for a,r in trds.iterrows():
#         col='red' if r['BuySell'] == -1 else 'green'
#         plt.plot([r['EntryDate'],r['ExitDate']],[r['EntryPrice'],r['ExitPrice']],color=col, marker='o')

    
    df=getDf(rec['Ticker'])
    df_l=df[(df.dt > startDt) & (df.dt < endDt)]

    
    data = [go.Ohlc(x=df_l['dt'],
                    open=df_l['o'],
                    high=df_l['h'],
                    low=df_l['l'],
                    close=df_l['c'])]
    
    layout = {
        'title': 'The Great Recession',
        'yaxis': {'title': 'Show trade'},
        'shapes': [{
            'x0': '2016-12-09', 'x1': '2016-12-09',
            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
            'line': {'color': 'rgb(30,30,30)', 'width': 1}
        }],
        'annotations': [{
            'x': '2016-12-09',  'xref': 'x', 'yref': 'paper',
            'showarrow': True, 'xanchor': 'left',
            'text': 'Increase Period Begins'
        }]
    }
    
    layout1 = {
        'title': f'{rec.Ticker} pnl {rec.Pnl/rec.EntryPrice*100} %',
        'yaxis': {'title': 'Price'},
        'annotations': ann(rec)
    }
    
    fig = dict(data=data, layout=layout1)
    py.iplot(fig)
        

        


showTrade(2)
showTrade(3)

ann(rec)



[{'x': Timestamp('2014-12-16 13:03:00'),
  'xref': 'x',
  'yref': 'paper',
  'showarrow': True,
  'xanchor': 'left',
  'text': 'sell'},
 {'x': Timestamp('2014-12-16 21:58:00'),
  'xref': 'x',
  'yref': 'paper',
  'showarrow': True,
  'xanchor': 'left',
  'text': 'buy'}]